In [1]:
import numpy as np
import aabb_interface
from triangle_intersection import py_triangle_triangle_intersection as triangles_intersection

In [2]:
# read mesh from off file
vertices, triangles = aabb_interface.read_mesh_from_off("data/eight.off")

Reading 315 vertices...
Reading 634 faces (only triangles will be stored)...
Successfully loaded mesh from data/eight.off


In [3]:
print(vertices.shape)

(315, 3)


In [4]:
print(triangles.shape)

(634, 3)


In [5]:
aabb_tree = aabb_interface.AABBNode()
aabb_tree.build_aabb_tree(vertices, triangles, 2)

In [6]:
tri_indices = aabb_tree.get_triangle_indices()
print(tri_indices)
print(aabb_tree.get_left())

[]


In [7]:
aabb_tree.get_bbox()

(array([-0.243695, -0.103765, -0.499314]),
 array([0.243695, 0.103828, 0.499314]))

In [8]:
aabb_tree.get_left().get_bbox()

(array([-0.243695, -0.103765, -0.499314]),
 array([0.242955, 0.103828, 0.045045]))

In [9]:
n = aabb_tree.get_left()

In [10]:
n = n.get_left()
print(n.get_bbox())
print(n.get_triangle_indices())

(array([-0.243695, -0.077763, -0.499314]), array([ 0.242955,  0.077821, -0.141   ]))
[]


In [11]:
V, F = aabb_interface.read_mesh_from_off("data/blobby.off")

Reading 2027 vertices...
Reading 4050 faces (only triangles will be stored)...
Successfully loaded mesh from data/blobby.off


In [12]:
test_tri = 2098
# test_tri = 1256
node_indexes = F[test_tri]
node_xyz = V[node_indexes, :]
print(node_xyz)
x_min, x_max = node_xyz[:, 0].min(), node_xyz[:, 0].max()
y_min, y_max = node_xyz[:, 1].min(), node_xyz[:, 1].max()
z_min, z_max = node_xyz[:, 2].min(), node_xyz[:, 2].max()

test_bbox = np.array([x_min, y_min, z_min, x_max, y_max, z_max])

[[-0.0976748 -0.0658008  0.166048 ]
 [-0.156829  -0.0700418  0.146298 ]
 [-0.103323  -0.097917   0.154292 ]]


In [13]:
aabb_tree.bbox_intersect(test_bbox)
aabb_tree.get_right().bbox_intersect(test_bbox)

True

In [14]:
def get_intersected_triangles(node, bbox):
    if not node.bbox_intersect(bbox):
        return []
    left_node = node.get_left()
    right_node = node.get_right()
    if left_node is None and right_node is None:
        return node.get_triangle_indices().tolist()
    else:
        return get_intersected_triangles(left_node, bbox) + get_intersected_triangles(right_node, bbox)


In [15]:
tri_indices = get_intersected_triangles(aabb_tree, test_bbox)
print(tri_indices)

[584, 591, 582, 589, 285, 590, 572, 588, 571, 574]


In [16]:
tri_xyz = V[F[test_tri, :], :]
print(tri_xyz)

[[-0.0976748 -0.0658008  0.166048 ]
 [-0.156829  -0.0700418  0.146298 ]
 [-0.103323  -0.097917   0.154292 ]]


In [17]:
for i_tri in tri_indices:
    inter_seg = triangles_intersection(tri_xyz, vertices[triangles[i_tri, :]])
    print("***", i_tri, inter_seg)

*** 584 []
*** 591 []
*** 582 []
*** 589 [[-0.13626948 -0.07818744  0.15016813]
 [-0.12544287 -0.07237193  0.15535135]]
*** 285 []
*** 590 [[-0.1461385  -0.07561127  0.1478952 ]
 [-0.13626948 -0.07818744  0.15016813]]
*** 572 [[-0.11334021 -0.06692392  0.16081774]
 [-0.11736747 -0.07062625  0.15841065]]
*** 588 [[-0.12544287 -0.07237193  0.15535135]
 [-0.11736747 -0.07062625  0.15841065]]
*** 571 []
*** 574 []


In [18]:
segments_list = []
for i in range(F.shape[0]):
    tri_xyz = V[F[i, :], :]
    tri_bbox = np.array([tri_xyz[:, 0].min(), tri_xyz[:, 1].min(), tri_xyz[:, 2].min(),
                          tri_xyz[:, 0].max(), tri_xyz[:, 1].max(), tri_xyz[:, 2].max()])
    for j in get_intersected_triangles(aabb_tree, tri_bbox):
        inter_seg = triangles_intersection(tri_xyz, vertices[triangles[j, :]])
        if len(inter_seg) == 2:
            segments_list.append((inter_seg[0, :], inter_seg[1, :]))

print(f"Done! Found {len(segments_list)} segments")
        
    

Done! Found 303 segments


In [19]:
import vtk
from vtk.util import numpy_support

# 1.1. Obtenir les points uniques et le mapping
all_points = np.unique(np.array([p for seg in segments_list for p in seg]), axis=0)
point_to_index = {tuple(p): i for i, p in enumerate(all_points)}

# --- 2. Création de vtkPoints ---
# Convertir le tableau NumPy de points en un vtkDataArray
vtk_points_data = numpy_support.numpy_to_vtk(all_points, deep=True)

# Créer un objet vtkPoints et lui assigner les données
vtk_points = vtk.vtkPoints()
vtk_points.SetData(vtk_points_data)

# --- 3. Création de vtkCellArray pour les lignes ---
vtk_lines = vtk.vtkCellArray()

# Insérer chaque segment comme une ligne (polyline) dans vtkCellArray
for p1, p2 in segments_list:
    idx1 = point_to_index[tuple(p1)]
    idx2 = point_to_index[tuple(p2)]
    
    # Créer l'objet vtkLine et définir les indices des points
    line = vtk.vtkLine()
    line.GetPointIds().SetId(0, idx1) # Index du point de départ
    line.GetPointIds().SetId(1, idx2) # Index du point de fin
    
    # Ajouter la ligne à vtkCellArray
    vtk_lines.InsertNextCell(line)

# --- 4. Création du maillage vtkPolyData ---
polydata = vtk.vtkPolyData()
polydata.SetPoints(vtk_points)
polydata.SetLines(vtk_lines)

# --- 5. Exportation en fichier VTK ---
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName('intersection_polyline.vtp')
writer.SetInputData(polydata)
writer.Write()

1

![title](image_result.png)